# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [42]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
city_data = "..\\WeatherPy\\weather_py_city_data.csv"
city_data_df = pd.read_csv(city_data)
city_data_df.head()


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tasiilaq,65.61,-37.64,20.70,96,100,6.64,GL,1605397932
1,Qaanaaq,77.48,-69.36,7.52,80,93,13.62,GL,1605397932
2,Ushuaia,-54.80,-68.30,42.80,79,75,9.37,AR,1605397932
3,Saskylakh,71.92,114.08,3.33,92,89,14.99,RU,1605397933
4,Norman Wells,65.28,-126.83,-5.80,83,75,2.24,CA,1605397867


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = city_data_df[['Latitude', 'Longitude']]
humidity = city_data_df['Humidity']

# Add Heatmap layer to map
figure_layout = {'width': '1200px',
    'border': '1px solid black',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, dissipating=False, point_radius=1)
fig.add_layer(heatmap_layer)
#fig.savefig("/Images/heat_map.png")
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', width='1200px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* Narrow down the DataFrame to find your ideal weather condition. For example:
* A max temperature lower than 85 degrees but higher than 70.
* Wind speed less than 15 mph.
* 30% cloudiness.

In [87]:
new_city_data = city_data_df.loc[(city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 85) & (city_data_df['Wind Speed'] < 15) & (city_data_df['Cloudiness'] <= 30)]
new_city_data_df = new_city_data.reset_index()
del new_city_data_df['index']
new_city_data_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,San Policarpo,12.18,125.51,82.74,81,22,10.36,PH,1605397936
1,Hilo,19.73,-155.09,78.80,65,1,6.93,US,1605397747
2,Acapulco de Juárez,16.86,-99.89,84.20,74,20,6.93,MX,1605397754
3,Korhogo,9.45,-5.63,72.34,37,2,1.16,CI,1605397984
4,Puerto Ayora,-0.74,-90.35,70.00,82,27,7.00,EC,1605398001
...,...,...,...,...,...,...,...,...,...
64,Kidal,18.44,1.41,70.83,23,0,10.76,ML,1605398206
65,Wagga Wagga,-35.12,147.37,73.40,46,0,10.29,AU,1605398206
66,Abu Dhabi,24.47,54.37,77.00,61,0,6.93,AE,1605398161
67,Jati,24.35,68.27,71.22,69,0,7.45,PK,1605398209


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
hotel_df = []
new_city_data_df['Hotel Name'] = ''
new_city_data_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,San Policarpo,12.18,125.51,82.74,81,22,10.36,PH,1605397936,
1,Hilo,19.73,-155.09,78.80,65,1,6.93,US,1605397747,
2,Acapulco de Juárez,16.86,-99.89,84.20,74,20,6.93,MX,1605397754,
3,Korhogo,9.45,-5.63,72.34,37,2,1.16,CI,1605397984,
4,Puerto Ayora,-0.74,-90.35,70.00,82,27,7.00,EC,1605398001,


In [98]:
cities = new_city_data_df.loc[new_city_data_df['City']]
lat = new_city_data_df.loc[new_city_data_df['Latitude']]
lng = new_city_data_df.loc[new_city_data_df['Longitude']]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
coordinates = f"{lat},{lng}"
params = {"location": coordinates,"types": "lodging","radius": 5000,"key": g_key}

for i in cities:

    try: 
        hotel_info = requests.get(base_url, params + i).json()
        print(f'Processing record')
        hotel_df.append(response_data["results"][0]["name"])
        
    except KeyError:
        print(f'No hotel near {city}. Skipped.')

KeyError: "None of [Index(['San Policarpo', 'Hilo', 'Acapulco de Juárez', 'Korhogo',\n       'Puerto Ayora', 'Pindiga', 'Bosaso', 'Gisborne', 'Port Lincoln',\n       'Neiafu', 'Santa Maria', 'George Town', 'Airai', 'Pochutla', 'Arlit',\n       'Praia', 'Mogadishu', 'Poum', 'Touros', 'Mandapeta', 'Iberia',\n       'Ulladulla', 'Lábrea', 'Isangel', 'Salalah', 'Kapoeta', 'Araouane',\n       'Charlestown', 'Batemans Bay', 'Chake Chake', 'Samarai', 'Sur',\n       'Harper', 'Vao', 'Tuscaloosa', 'Makkah al Mukarramah', 'Ikorodu',\n       'Maragogi', 'Nelson Bay', 'Chicomuselo', 'Atoyac', 'Kafue',\n       'Banjarmasin', 'La Estancia', 'Vila Velha', 'Tefé', 'Karratha',\n       'Youkounkoun', 'Oussouye', 'Oistins', 'Cayenne', 'Koumac', 'Teknāf',\n       'Gouyave', 'Dubbo', 'Kupang', 'Puri', 'Jardim', 'Casino', 'Hilton Head',\n       'La Cruz', 'San Vicente', 'Nísia Floresta', 'Pine Bluff', 'Kidal',\n       'Wagga Wagga', 'Abu Dhabi', 'Jati', 'Umluj'],\n      dtype='object')] are in the [index]"

In [100]:
# Could  not get loop to work :(

In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
